Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Automated Machine Learning
**Demand Forecasting Using Many Models**

## Contents
1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train Many Models](#TrainManyModels)
1. [Train Baseline](#TrainBaseline)
1. [Test Set Inference](#TestSetInference)
1. [Test Set Evaluation](#TestSetEvaluation)
1. [Generate Forecast](#GenerateForecast)
1. [Schedule Inference Pipelines](#ScheduleInference)

## 1. Introduction

The objective of this notebook is to illustrate how to use the AutoML many models solution accelertor for demand forecasting tasks. It walks you through all stages of model evaluation and production process starting with data ingestion and concluding with scheduling inference runs.

We use a subset of UCI electricity data ([link](https://archive.ics.uci.edu/ml/datasets/ElectricityLoadDiagrams20112014#)) with the objective of predicting electricity demand per consumer 24 hours ahead. The data was preprocessed using the [data prep notebook](https://github.com/Azure/azureml-examples/blob/main/v1/python-sdk/tutorials/automl-with-azureml/forecasting-data-preparation/auto-ml-forecasting-data-preparation.ipynb) notebook. Please refer to it for illustration on how to download the data from the source, aggregate to an hourly frequency, convert from wide to long format and upload to the Datastore. Here, we will work with the already uploaded data. 

Having a problem description such as to generate accurate forecasts 24 hours ahead sounds like a relatively straight forward task. However, there are quite a few steps a user needs to take before the model is put in production. A user needs to prepare the data, partition it into appropriate sets, select the best model, evaluate it against a baseline, and monitor the model in real life to collect enough observations on how it would perform had it been put in production. Some of these steps are time consuming, some require certain expertise in writing code. The steps shown in this notebook follow a typical thought process one follows before the model is put in production.

Make sure you have executed the [configuration](https://github.com/Azure/MachineLearningNotebooks/blob/master/configuration.ipynb) before running this notebook.

## 1. Setup

In [ ]:
import json
import logging
import os

from matplotlib import pyplot as plt
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig

This sample notebook may use features that are not available in previous versions of the Azure ML SDK.

In [ ]:
print("This notebook was created using version 1.47.0 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

Accessing the Azure ML workspace requires authentication with Azure.

The default authentication is interactive authentication using the default tenant. Executing the ws = Workspace.from_config() line in the cell below will prompt for authentication the first time that it is run.

If you have multiple Azure tenants, you can specify the tenant by replacing the ws = Workspace.from_config() line in the cell below with the following:
```
from azureml.core.authentication import InteractiveLoginAuthentication
auth = InteractiveLoginAuthentication(tenant_id = 'mytenantid')
ws = Workspace.from_config(auth = auth)
```
If you need to run in an environment where interactive login is not possible, you can use Service Principal authentication by replacing the ws = Workspace.from_config() line in the cell below with the following:
```
from azureml.core.authentication import ServicePrincipalAuthentication
auth = ServicePrincipalAuthentication('mytenantid', 'myappid', 'mypassword')
ws = Workspace.from_config(auth = auth)
```
For more details, see aka.ms/aml-notebook-auth

In [ ]:
import datetime
import uuid

ws = Workspace.from_config()
datastore = ws.get_default_datastore()

# Choose a name for the run history container in the workspace.
experiment_name = "forecasting-many-models-" + datetime.datetime.now().strftime(
    "%Y%m%d"
)
experiment = Experiment(ws, experiment_name)
experiment = Experiment(ws, experiment_name)

output = {}
output["Subscription ID"] = ws.subscription_id
output["Workspace"] = ws.name
output["Resource Group"] = ws.resource_group
output["Location"] = ws.location
output["Run History Name"] = experiment_name
pd.set_option("display.max_colwidth", None)
outputDf = pd.DataFrame(data=output, index=[""])
outputDf.T

### 2.1. Compute 

#### Create or Attach existing AmlCompute

You will need to create a compute target for your AutoML run. In this tutorial, you will create AmlCompute as your training compute resource.

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.


To run deep learning models we recommend to use GPU compute. Here, we use a 12 node cluster of the `Standard_NC8as_T4_v3` [series](https://learn.microsoft.com/en-us/azure/virtual-machines/nct4-v3-series) for illustration purposes. You will need to adjust the compute type and the number of nodes based on your needs which can be driven by the speed needed for model seelction, data size, etc. 

#### Creation of AmlCompute takes approximately 5 minutes. 
If the AmlCompute with that name is already in your workspace, this code will skip the creation process.
As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
amlcompute_cluster_name = "demand-fcst-mm-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_DS15_V2", max_nodes=5
    )
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment

aml_run_config = RunConfiguration()
aml_run_config.target = compute_target

USE_CURATED_ENV = True
if USE_CURATED_ENV:
    curated_environment = Environment.get(
        workspace=ws, name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu"
    )
    aml_run_config.environment = curated_environment
else:
    aml_run_config.environment.python.user_managed_dependencies = False

    # Add some packages relied on by data prep step
    aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
        conda_packages=["pandas", "scikit-learn"],
        pip_packages=["azureml-sdk", "azureml-dataset-runtime[fuse,pandas]"],
        pin_sdk_version=False,
    )

## 3. Data
If you ran the data [preparation notebook](https://github.com/Azure/azureml-examples/blob/main/v1/python-sdk/tutorials/automl-with-azureml/forecasting-data-preparation/auto-ml-forecasting-data-preparation.ipynb) and want to use the registered data, skip section 3.1 and, instead, uncomment and execute the code in section 3.2. If, on the other hand, you did not run the notebook and want to use the data that we pre-processed and saved in the public blob, execute the code in section 3.1.

### 3.1 Loading and registering the data from public blob store

Run the code in this section only if you want to use the data that is already available in the blobstore. If you want to use your own data that is already registered in your workspace, skip this section and procceed to run the commented out code in section 3.2.

The following code registers a datastore `autom_fcst_many_models` in your workspace and links the data from the container `automl-sample-notebook-data`.

In [ ]:
from azureml.core import Datastore

# Please change the following to point to your own blob container and pass in account_key
blob_datastore_name = "autom_fcst_many_models"
container_name = "automl-sample-notebook-data"
account_name = "automlsamplenotebookdata"

print(f'Creating datastore "{blob_datastore_name}" in your workspace ...\n---')
demand_mm_datastore = Datastore.register_azure_blob_container(
    workspace=ws,
    datastore_name=blob_datastore_name,
    container_name=container_name,
    account_name=account_name,
    create_if_not_exists=True,
)

The following code registers datasets from the `automl-sample-notebook-data` container in the datastore we just created. Once the datasets are registered, we will be able to use them in our experiments.

In [ ]:
from azureml.core import Dataset

print("Registering datasets in your workspace ...\n---")

FOLDER_PREFIX_NAME = "uci_electro_small_public_mm"

target_path_train = f"{FOLDER_PREFIX_NAME}_train"
target_path_test = f"{FOLDER_PREFIX_NAME}_test"
target_path_inference = f"{FOLDER_PREFIX_NAME}_infer"

train_dataset = Dataset.Tabular.from_delimited_files(
    path=demand_mm_datastore.path(target_path_train + "/"),
    validate=False,
    infer_column_types=True,
).register(workspace=ws, name=target_path_train, create_new_version=True)

test_dataset = Dataset.Tabular.from_delimited_files(
    path=demand_mm_datastore.path(target_path_test + "/"),
    validate=False,
    infer_column_types=True,
).register(workspace=ws, name=target_path_test, create_new_version=True)

inference_dataset = Dataset.Tabular.from_delimited_files(
    path=demand_mm_datastore.path(target_path_inference + "/"),
    validate=False,
    infer_column_types=True,
).register(workspace=ws, name=target_path_inference, create_new_version=True)

### 3.2 Using data that is registered in your workspace

If you ran the [data prep notebook](https://github.com/Azure/azureml-examples/blob/main/v1/python-sdk/tutorials/automl-with-azureml/forecasting-data-preparation/auto-ml-forecasting-data-preparation.ipynb) notebook, the train, test and inference sets are already uploaded and registered in your workspace. Uncomment the following code and change the `DATASET_PREFIX_NAME`, to match the value in the data preparation notebook, and run the code.

In [ ]:
# from azureml.data.dataset_factory import TabularDatasetFactory
# from azureml.core.dataset import Dataset

# DATASET_PREFIX_NAME = "uci_electro_small_mm"
# print(f'Dataset prefix name: {DATASET_PREFIX_NAME}\n---\nLoading train, validation, test and inference sets ...\n---')

# target_path_train = f"{DATASET_PREFIX_NAME}_train"
# target_path_test = f"{DATASET_PREFIX_NAME}_test"
# target_path_inference = f"{DATASET_PREFIX_NAME}_inference"

# train_dataset = Dataset.get_by_name(ws, name=target_path_train)
# test_dataset = Dataset.get_by_name(ws, name=target_path_test)
# inference_dataset = Dataset.get_by_name(ws, name=target_path_inference)

### 3.3 Test and inference sets

Note that we have *test* and *inference* sets. The difference between the two is the presence of the target column. The test set contains the target column and is used to evaluate model performance using [rolling forecast](https://learn.microsoft.com/en-us/azure/machine-learning/v1/how-to-auto-train-forecast-v1#evaluating-model-accuracy-with-a-rolling-forecast). On the other hand, the target column is not present in the inference set to illustrate how to generate an actual forecast.

In [ ]:
print("The first few rows of the test set ...\n---")
print(test_dataset.take(5).to_pandas_dataframe())

In [ ]:
print("The first few rows of the inference set ...\n---")
print(inference_dataset.take(5).to_pandas_dataframe())

Let's set up what we know about the dataset.

- **Target column** is what we want to forecast. In our case it is electricity consumption per customer measured in kilowatt hours (kWh).
- **Time column** is the time axis along which to predict.
- **Time series identifier columns** are identified by values of the columns listed `time_series_id_column_names`. In our case all unique time series are identified by a single column `customer_id`. However, it is quite common to have multiple columns identifying unique time series. See the [link](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-auto-train-forecast#configuration-settings) for a more detailed explanation on this topic.

In [ ]:
target_column_name = "usage"
time_column_name = "datetime"
GRAIN_COL = "customer_id"
time_series_id_column_names = [GRAIN_COL]

Next, we download training data from the Datastore to make sure it looks as expected. If your dataset is large, there is no need to store it in the memory. In this case, skip the next block of code.

In [ ]:
train_df = train_dataset.to_pandas_dataframe()

nseries = train_df.groupby(time_series_id_column_names).ngroups
print(
    f"Data contains {nseries} individual time-series:\n{list(train_df[GRAIN_COL].unique())}\n---"
)
print("Printing the first few rows of the training data ...\n---")
train_df.head()

## 4. Train Many Models

In this section we will train many models as well as the baseline model. The baseline model will be used as a reference point to understand many models' accuracy performance. The goal of forecasting is to have the most accurate predictions measured by some accuracy metric. What is considered an accurate prediction is fairly subjective. Take, for example, the MAPE (mean absolute percentage error) metric. A perfect forecast will result in the MAPE value of zero, which is not achievable using business data. For this reason it is imperative to have a baseline model to compare AutoML results against. Doing this adds objectivity to the model acceptance criteria. 

The baseline model can be the model that is currently in production. Oftentimes, the baseline is set to be a Naive forecast, which we will use in this notebook. The choice of the baseline is also specific to the data. For example, if there is a clear trend in the data one may not want to use a Naive model.  Instead, one can use an ARIMA model. Please see this [document](https://learn.microsoft.com/en-us/azure/machine-learning/v1/how-to-configure-auto-train-v1#supported-models) for a list of AutoML models one can chose from to use as a baseline model.

The following 2 parameters allow us to re-use training runs for the many-models and baseline models, respectively. This can be helpful it you need to experiment with the post model training steps thus avoiding the need to kick off a new training run which can be computationally expensive.

In [ ]:
IS_MANY_MODELS_TRAINED = False
IS_BASE_MODEL_TRAINED = False

### 4.1 Train AutoML model

#### 4.1.1 Set up training parameters
We need to provide the `ForecastingParameters`, `AutoMLConfig` and `ManyModelsTrainParameters` objects. For the forecasting task we also need to define several settings including the name of the time column, the maximum forecast horizon, and the partition column name(s) definition.

#### Forecasting Parameters
To define forecasting parameters for your experiment training, you can leverage the `ForecastingParameters` class. The table below details the forecasting parameters we will be passing into our experiment.


|Property|Description|
|-|-|
|**time_column_name**|The name of the time column in the data.|
|**forecast_horizon**|The forecast horizon is how many periods forward you would like to forecast. This integer horizon is in units of the timeseries frequency (e.g. daily, weekly).|
|**time_series_id_column_names**|The column names used to uniquely identify the time series in data that has multiple rows with the same timestamp. If the time series identifiers are not defined, the data set is assumed to be one time series.|
| **cv_step_size**| Number of periods between two consecutive cross-validation folds. The default value is "auto", in which case AutoMl determines the cross-validation step size automatically, if a validation set is not provided. Or users could specify an integer value. |
|**freq**|Forecast frequency. This optional parameter represents the period for which the forecast is desired, for example, daily, weekly, yearly, etc. Use this parameter for the correction of time series containing irregular data points or for padding of short time series. The frequency needs to be a pandas offset alias. Please refer to [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects) for more information.


#### AutoMLConfig arguments
|Property|Description|
|-|-|
| **task**                           | forecasting |
| **primary_metric**                 | This is the metric that you want to optimize. Forecasting supports the following primary metrics<ul><li>`normalized_root_mean_squared_error`</li><li>`normalized_mean_absolute_error`</li><li>`spearman_correlation`</li><li>`r2_score`</li></ul> We recommend using either the normalized root mean squared error or normalized mean absolute erorr as a primary metric because they measure forecast accuracy. See the [link](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-automl-forecasting-faq#how-do-i-choose-the-primary-metric) for a more detailed discussion on this topic. |
| **experiment_timeout_hours**       | Maximum amount of time in hours that each experiment can take before it terminates. This is optional but provides customers with greater control on exit criteria. When setting this criteria we advise to take into account the number of desired iterations parameter and set experiment timeout setting such that the desired number of iterations will be completed.|
| **iterations**                     | Number of models to train. This is optional but provides customers with greater control on exit criteria. |
| **label_column_name**              | The name of the target column we are trying to predict. |
| **n_cross_validations**            | Number of cross validation splits. The default value is "auto", in which case AutoMl determines the number of cross-validations automatically. Or users could specify an integer value. Rolling Origin Validation is used to split time-series in a temporally consistent way.|
| **enable_early_stopping**          | Flag to enable early termination if the primary metric is no longer improving. |
| **blocked_models**                 | List of models we want to block. For illustration purposes and to reduce the runtime, we block all time series specific models. The defaule value is None or an empty list.|

#### ManyModelsTrainParameters arguments
|Property|Description|
|-|-|
| **automl_settings** | The `AutoMLConfig` object defined above. |
| **partition_column_names** | The names of columns used to group your models. For timeseries, the groups must not split up individual time-series. That is, each group must contain one or more whole time-series. |

Note, that the `time_series_id_column_names` and `partition_column_names` do not have to be the same. In our scenario, they are the same since we are interested in training one model per customer. We have 10 customers in our dataset and there will be 10 models trained. Say, you decide to cluster customers into groups, for example, each group has 2 customers and you want to train one model per group of customers. In such scenario, you will partition the data by groups, and the `time_series_id_column_names` will be different from the `partition_column_names`.

In [ ]:
from azureml.automl.core.forecasting_parameters import ForecastingParameters
from azureml.train.automl.automlconfig import AutoMLConfig
from azureml.train.automl.runtime._many_models.many_models_parameters import (
    ManyModelsTrainParameters,
)

forecast_horizon = 24
partition_column_names = time_series_id_column_names

BLOCKED_MODELS = [
    "Naive",
    "SeasonalNaive",
    "Average",
    "SeasonalAverage",
    "Prophet",
    "ExponentialSmoothing",
    "ExtremeRandomTrees",
    "AutoArima",
    "Arimax",
]
EXPERIMENT_TIMEOUT_HOURS = 1

forecasting_parameters = ForecastingParameters(
    time_column_name=time_column_name,
    forecast_horizon=forecast_horizon,
    time_series_id_column_names=time_series_id_column_names,
    cv_step_size="auto",
    freq="H",
)

automl_settings = AutoMLConfig(
    task="forecasting",
    primary_metric="normalized_root_mean_squared_error",
    iteration_timeout_minutes=20,
    iterations=25,
    experiment_timeout_hours=EXPERIMENT_TIMEOUT_HOURS,
    label_column_name=target_column_name,
    n_cross_validations="auto",  # Feel free to set to a small integer (>=2) if runtime is an issue.
    blocked_models=BLOCKED_MODELS,
    track_child_runs=False,
    forecasting_parameters=forecasting_parameters,
)

mm_paramters = ManyModelsTrainParameters(
    automl_settings=automl_settings, partition_column_names=partition_column_names
)

### Set up many models pipeline

Parallel run step is leveraged to train multiple models at once. To configure the ParallelRunConfig you will need to determine the appropriate number of workers and nodes for your use case. The `process_count_per_node` is based off the number of cores of the compute VM. The node_count will determine the number of master nodes to use, increasing the node count will speed up the training process.

| Property                           | Description|
|-|-|
| **experiment**                     | The experiment used for training. |
| **train_data**                     | The file dataset to be used as input to the training run. |
| **node_count**                     | The number of compute nodes to be used for running the user script. We recommend to start with 3 and increase the node_count if the training time is taking too long. |
| **process_count_per_node**         | Process count per node, we recommend 2:1 ratio for number of cores: number of processes per node. eg. If node has 16 cores then configure 8 or less process count per node for optimal performance. |
| **train_pipeline_parameters**      | The set of configuration parameters defined in the previous section. |
| **run_invocation_timeout**         | Maximum amount of time in seconds that the `ParallelRunStep` class is allowed. This is optional but provides customers with greater control on exit criteria. This must be greater than `experiment_timeout_hours` by at least 300 seconds. Here, we we add a buffer of 1000 seconds. |
| **arguments**                      | Arguments to be passed to training script. Here, we pass the parameter `retrain_failed_models` and set it to True. If training a model for any partition fails, AutoML will kick off a new child run for that partition.|

**Note**: Total time it takes for the **training step** in the pipeline to complete  equals to 

$$
\left( \frac{t}{ p \times n } \right) \times k
$$

where
- $ t $ is time it takes to train one partition (can be viewed in the training logs)
- $ p $ is the process count per node
- $ n $ is the node count
- $ k $ is total number of partitions in time series based on `partition_column_names`

In [ ]:
from azureml.contrib.automl.pipeline.steps import AutoMLPipelineBuilder


training_pipeline_steps = AutoMLPipelineBuilder.get_many_models_train_steps(
    experiment=experiment,
    train_data=train_dataset,
    compute_target=compute_target,
    node_count=5,
    process_count_per_node=2,
    run_invocation_timeout=(EXPERIMENT_TIMEOUT_HOURS * 3600 + 1000),
    train_pipeline_parameters=mm_paramters,
    arguments=["--retrain_failed_models", "True"],
)

Note, the output of the previous cell prints out the name of the partitioned dataset. This allows you to run a new experiment on the already partiitoned dataset. What this does is it skips a data partitioning step, and reduces the runtime. To use already partioned dataset, uncomment and execute the following code _**before**_ building the `training_pipeline_steps`.

In [ ]:
# from azureml.data.dataset_factory import TabularDatasetFactory
# from azureml.core.dataset import Dataset

# PARTITIONED_TRAIN_DATASET_NAME = "<Paste the output of the previous window here.>"
# train_dataset = Dataset.get_by_name(ws, name=PARTITIONED_TRAIN_DATASET_NAME)

### Submit the pipeline to run
Next we submit our pipeline to run. The whole training pipeline takes about 20 minutes on a 5 node STANDARD_DS15_V2  cluster.

In [ ]:
from azureml.pipeline.core import Pipeline

training_pipeline = Pipeline(ws, steps=training_pipeline_steps)

In [ ]:
print(f"Are many models trained? {IS_MANY_MODELS_TRAINED}\n---")

In [ ]:
if not IS_MANY_MODELS_TRAINED:
    print("Training new AutoML model ...\n---")
    training_run = experiment.submit(training_pipeline)
else:
    from azureml.train.automl.run import AutoMLRun
    from azureml.pipeline.core.run import PipelineRun

    PIPELINE_RUN_ID = "<Replace with the PipelineRunId used for training many models>"  # Copy the output of Submitted PipelineRun to re-use trained models
    training_run = PipelineRun(experiment=experiment, run_id=PIPELINE_RUN_ID)
    print(f"Using previously trained model. Pipeline run ID: {PIPELINE_RUN_ID}\n---")

In [ ]:
if not IS_MANY_MODELS_TRAINED:
    training_run.wait_for_completion(show_output=False)
    IS_MODEL_TRAINED = True

In [ ]:
training_run

## 5. Train the baseline model

We will use Naive model as our baseline. To train it, we kick off another automl experiment with the following settings. Please note that we added the Naive model to the allowed models list, and set the number of iterations to 1 since we only interested in training one specific model. To reduce the training time, we set the number of cross validations to 2. Read the following [document](https://learn.microsoft.com/en-us/azure/machine-learning/v1/how-to-auto-train-forecast-v1#training-and-validation-data) for more information on this topic.

The only `AutoMLConfig` settings you might consider changing are the `experiment_timeout_hours` and `allowed_models`. You might want to increase the experiment timeout if your data has lots of unique time series. The allowed model list can be modified to refect a different choice of the baseline model and can be selected from the supported [forecasting models](https://learn.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.constants.supportedmodels.forecasting) and [regression models](https://learn.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.constants.supportedmodels.regression).

In [ ]:
from azureml.automl.core.forecasting_parameters import ForecastingParameters

forecasting_parameters = ForecastingParameters(
    time_column_name=time_column_name,
    forecast_horizon=forecast_horizon,
    time_series_id_column_names=time_series_id_column_names,
    cv_step_size=1,
    freq="H",
)

automl_config = AutoMLConfig(
    task="forecasting",
    debug_log="baseline.log",
    primary_metric="normalized_root_mean_squared_error",
    experiment_timeout_hours=1,
    iterations=1,
    training_data=train_dataset,
    label_column_name=target_column_name,
    compute_target=compute_target,
    enable_early_stopping=True,
    n_cross_validations=2,
    verbosity=logging.INFO,
    max_cores_per_iteration=-1,
    enable_dnn=False,
    allowed_models=["Naive"],
    forecasting_parameters=forecasting_parameters,
)

In [ ]:
print(f"Is the baseline model trained? {IS_BASE_MODEL_TRAINED}\n---")

In [ ]:
IS_BASE_MODEL_TRAINED = False

In [ ]:
if not IS_BASE_MODEL_TRAINED:
    remote_base_run = experiment.submit(automl_config, show_output=False)
else:
    from azureml.train.automl.run import AutoMLRun

    BASE_RUN_ID = "<Replace with the run ID used for training the baseline model>"
    # during the initial training run copy-paste the run id to be utilized later if needed.
    remote_base_run = AutoMLRun(experiment=experiment, run_id=BASE_RUN_ID)
    print(f"Using previously trained model. Run ID: {BASE_RUN_ID}\n---")

In [ ]:
if not IS_BASE_MODEL_TRAINED:
    remote_base_run.wait_for_completion(show_output=False)
    IS_BASE_MODEL_TRAINED = True

## 6. Test set inference 

### 6.1 Many models inferences

We create an output folder which will be used to save the output of our experiments.

In [ ]:
# create an output folder
OUTPUT_DIR = os.path.join(os.getcwd(), "forecast_output")
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
test_experiment = Experiment(ws, experiment_name + "_inference")
test_experiment_base = Experiment(ws, experiment_name + "_inference_base")

#### Set up output dataset for inference data
Output of inference can be represented as [OutputFileDatasetConfig](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.output_dataset_config.outputdatasetconfig?view=azure-ml-py) object and OutputFileDatasetConfig can be registered as a dataset. 

In [ ]:
from azureml.data import OutputFileDatasetConfig

output_test_data_ds = OutputFileDatasetConfig(
    name="many_models_inference_output",
    destination=(datastore, "uci_electro_small/test_set_output/"),
).register_on_complete(name="uci_electro_small_test_data_ds")

For many models we need to provide the ManyModelsInferenceParameters object.

#### `ManyModelsInferenceParameters` arguments
| Property                           | Description|
| :---------------                   | :------------------- |
| **partition_column_names**         | List of column names that identifies groups. |
| **target_column_name**             | \[Optional\] Column name only if the inference dataset has the target. |
| **time_column_name**               | \[Optional\] Time column name only if it is timeseries. |
| **inference_type**                 | \[Optional\] Which inference method to use on the model. Possible values are 'forecast', 'predict_proba', and 'predict'. |
| **forecast_mode**                  | \[Optional\] The type of forecast to be used, either 'rolling' or 'recursive'; defaults to 'recursive'. |
| **step**                           | \[Optional\] Number of periods to advance the forecasting window in each iteration **(for rolling forecast only)**; defaults to 1. |

#### `get_many_models_batch_inference_steps` arguments
| Property                           | Description|
| :---------------                   | :------------------- |
| **experiment**                     | The experiment used for inference run. |
| **inference_data**                 | The data to use for inferencing. It should be the same schema as used for training.
| **compute_target**                 | The compute target that runs the inference pipeline. |
| **node_count**                     | The number of compute nodes to be used for running the user script. We recommend to start with the number of cores per node (varies by compute sku). |
| **process_count_per_node**         | \[Optional\] The number of processes per node. By default it's 2 (should be at most half of the number of cores in a single node of the compute cluster that will be used for the experiment).
| **inference_pipeline_parameters**  | \[Optional\] The `ManyModelsInferenceParameters` object defined above. |
| **append_row_file_name**           | \[Optional\] The name of the output file (optional, default value is 'parallel_run_step.txt'). Supports 'txt' and 'csv' file extension. A 'txt' file extension generates the output in 'txt' format with space as separator without column names. A 'csv' file extension generates the output in 'csv' format with comma as separator and with column names. |
| **train_run_id**                   | \[Optional\] The run id of the **training pipeline**. By default it is the latest successful training pipeline run in the experiment. |
| **train_experiment_name**          | \[Optional\] The train experiment that contains the train pipeline. This one is only needed when the train pipeline is not in the same experiement as the inference pipeline. |
| **run_invocation_timeout**         | \[Optional\] Maximum amount of time in seconds that the `ParallelRunStep` class is allowed. This is optional but provides customers with greater control on exit criteria. |
| **output_datastore**               | \[Optional\] The `Datastore` or `OutputDatasetConfig` to be used for output. If specified any pipeline output will be written to that location. If unspecified the default datastore will be used. |
| **arguments**                      | \[Optional\] Arguments to be passed to inference script. Possible argument is '--forecast_quantiles' followed by quantile values. |

In [ ]:
from azureml.contrib.automl.pipeline.steps import AutoMLPipelineBuilder
from azureml.train.automl.runtime._many_models.many_models_parameters import (
    ManyModelsInferenceParameters,
)

output_file_name = "parallel_run_step.csv"

mm_parameters = ManyModelsInferenceParameters(
    partition_column_names=time_series_id_column_names,
    time_column_name=time_column_name,
    target_column_name=target_column_name,
)

inference_steps = AutoMLPipelineBuilder.get_many_models_batch_inference_steps(
    experiment=experiment,
    inference_data=test_dataset,
    node_count=2,
    process_count_per_node=8,
    compute_target=compute_target,
    run_invocation_timeout=300,
    output_datastore=output_test_data_ds,
    train_run_id=training_run.id,
    train_experiment_name=training_run.experiment.name,
    inference_pipeline_parameters=mm_parameters,
    append_row_file_name=output_file_name,
)

In [ ]:
from azureml.pipeline.core import Pipeline

inference_pipeline = Pipeline(ws, steps=inference_steps)

In [ ]:
inference_run = experiment.submit(inference_pipeline)
inference_run.wait_for_completion(show_output=False)

### 6.2 Retreive the test set predictions from the many models

The forecasting pipeline forecasts the orange juice quantity for a Store by Brand. The pipeline returns one file with the predictions for each store and outputs the result to the forecasting_output Blob container. The details of the blob container is listed in 'forecasting_output.txt' under Outputs+logs. 

The following code snippet:
1. Downloads the contents of the output folder that is passed in the parallel run step 
2. Reads the output file that has the predictions as pandas dataframe 
3. Displays the top 5 rows of the predictions

In [ ]:
# from azureml.pipeline.core import StepRun
# ! pip show azureml-pipeline-core

In [ ]:
from azureml.contrib.automl.pipeline.steps.utilities import get_output_from_mm_pipeline

forecasting_results_name = "forecasting_results"
forecasting_output_name = "many_models_inference_output"
forecast_file = get_output_from_mm_pipeline(
    inference_run, forecasting_results_name, forecasting_output_name, output_file_name
)
df = pd.read_csv(forecast_file)
print(
    "Prediction has ", df.shape[0], " rows. Here the first 5 rows are being displayed."
)
df.head(5)

In [ ]:
df.rename(columns={"Predictions": "predicted"}, inplace=True)
df.to_csv(os.path.join(OUTPUT_DIR, "test-set-predictions-many-models.csv"), index=False)
df.head(5)

### 6.3 Inference the baseline model

Next, we perform a rolling evaluation on the test set for the baseline model. To do this, we use the `run_remote_inference` method which downloads the pickle file of the model into the temporary folder `forecast_naive` and copies the `inference_script_naive.py` file to it. This folder is then uploaded on the compute cluster where inference is performed. The `inference_script_naive.py` script performs a rolling evaluation on the test set, similarly to what we have done in section 6.2. Upon completion of this step, we delete the newly created `forecast_naive` folder.

In [ ]:
baseline_run = remote_base_run.get_best_child()
baseline_model_name = baseline_run.properties["model_name"]
baseline_run

In [ ]:
import shutil
from scripts.helper_scripts import run_remote_inference_naive

if True:
    remote_base_run_test = run_remote_inference_naive(
        test_experiment=test_experiment_base,
        compute_target=compute_target,
        train_run=baseline_run,
        test_dataset=test_dataset,
        target_column_name=target_column_name,
        rolling_evaluation_step_size=forecast_horizon,
        inference_folder="./forecast_naive",  # needed otherwise it will be looking for DNN environment b/c/ model.pt is uploaded to the clsuter for inference
    )
    remote_base_run_test.wait_for_completion(show_output=False)

    # download the forecast file to the local machine
    print("Downloading test data with prediction ...\n---")
    remote_base_run_test.download_file(
        "outputs/predictions.csv",
        os.path.join(OUTPUT_DIR, "test-set-predictions-base.csv"),
    )

    # delete downloaded scripts
    print("Removing auxiliary files ...\n---")
    shutil.rmtree("./forecast_naive")

## 7. Test set model evaluation

In this section we will evaluate the test set performance for many models and compare it with the baseline. We will generate time series plots for forecasts and actuals, calculate accuracy metrics and plot the evolution of metrics for each model over time. All output from this section will be stored in the `forecast_output` folder and can be referenced any time you need it.

### 7.1 Load test set results

Here, we will import test set results for both many-models and baseline experiments.

In [ ]:
backtest_automl = pd.read_csv(
    os.path.join(OUTPUT_DIR, "test-set-predictions-many-models.csv"),
    parse_dates=[time_column_name],
)
backtest_base = pd.read_csv(
    os.path.join(OUTPUT_DIR, "test-set-predictions-base.csv"),
    parse_dates=[time_column_name],
)

Nex, we combine outputs into a single dataframe which will be used for plotting and scoring.

In [ ]:
backtest = backtest_automl.merge(
    backtest_base.drop(target_column_name, axis=1),
    on=["customer_id", "datetime"],
    how="inner",
    suffixes=["", "_base"],
)
backtest.head()

In [ ]:
print(
    f"N model: {backtest_automl.shape[0]}. N baseline: {backtest_base.shape[0]}. N merged: {backtest.shape[0]}"
)

The data we are working with has an hourly frequency and we plan to generate the forecasts every 24 hours. If the model were to be put in production such that the forecasts are generated and model's performance is monitored every 24 hours, we will mimic the scoring process on the test set by generating daily accuracy metrics. To do this, we create a date column ("ymd"). If you want to score the output at any other frequency, say, weekly, just change the frequency parameter to the desired frequency.

In [ ]:
PERIOD_COLUMN = "ymd"

backtest[PERIOD_COLUMN] = backtest[time_column_name].dt.to_period(
    "D"
)  # year-month-day to be used for daily metrics computation
backtest.head()

### 7.2 Generate time series plots

Here, we generate forecast versus actuals plot for the test set for both the best many models and the baseline. Since we use rolling evaluation with the step size of 24 hours, this mimics the behavior of putting both models in production and monitoring their behavior for the duration of the test set. This step allows users to make informed decisons about model performance and saves numerous costs associated with productionalizing the model and monitoring its performance in real life. 

In [ ]:
from scripts.helper_scripts import _draw_one_plot
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

plot_filename = "forecast_vs_actual.pdf"

pdf = PdfPages(os.path.join(os.getcwd(), OUTPUT_DIR, plot_filename))
for _, one_forecast in backtest.groupby(GRAIN_COL):
    one_forecast[time_column_name] = pd.to_datetime(one_forecast[time_column_name])
    one_forecast.sort_values(time_column_name, inplace=True)
    _draw_one_plot(
        one_forecast,
        time_column_name,
        target_column_name,
        [GRAIN_COL],
        [target_column_name, "predicted", "predicted_base"],
        pdf,
        plot_predictions=True,
    )
pdf.close()

### 7.3 Calculate metrics
Here, we will calculate the metric of interest for each day. For illustration purposes we use root mean squared error as the metric of choice. However, the `compute_all_metrics` method calculated all primary and secondary metrics for AutoML runs. Please refer to this <u>*Regression/forecasting metrics*</u> section in this [document](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-understand-automated-ml#regressionforecasting-metrics) for the list of available metrics. We will calculate the distribution of this metric for each time series in our dataset. Looking at the descrptive stats of such metrics can be more informative than calculating a single metric such as the mean for each time series. As an example, we are looking at the RMSE (root mean squared error) metric, but you can choose any other metric computed.

In [ ]:
from scripts.helper_scripts import compute_all_metrics

metrics_per_grain_day = compute_all_metrics(
    fcst_df=backtest,
    actual_col=target_column_name,
    fcst_col="predicted",
    ts_id_colnames=[GRAIN_COL, PERIOD_COLUMN],
)

In [ ]:
print(
    f'List of available metrics: {metrics_per_grain_day["metric_name"].unique()}\n---'
)

In [ ]:
DESIRED_METRIC_NAME = "root_mean_squared_error"

metrics_per_grain_day = metrics_per_grain_day.query(
    f'metric_name == "{DESIRED_METRIC_NAME}"'
)
metrics_per_grain_day[[GRAIN_COL, PERIOD_COLUMN]] = metrics_per_grain_day[
    "time_series_id"
].str.split("|", 1, expand=True)
metrics_per_grain_day.to_csv(
    os.path.join(OUTPUT_DIR, "metrics-automl.csv"), index=False
)
metrics_per_grain_day.groupby(GRAIN_COL)["metric"].describe()

In [ ]:
# baseline metrics
metrics_per_grain_day_base = compute_all_metrics(
    fcst_df=backtest,
    actual_col=target_column_name,
    fcst_col="predicted_base",
    ts_id_colnames=[GRAIN_COL, PERIOD_COLUMN],
)
metrics_per_grain_day_base = metrics_per_grain_day_base.query(
    f'metric_name == "{DESIRED_METRIC_NAME}"'
)
metrics_per_grain_day_base[[GRAIN_COL, PERIOD_COLUMN]] = metrics_per_grain_day[
    "time_series_id"
].str.split("|", 1, expand=True)
metrics_per_grain_day_base.to_csv(
    os.path.join(OUTPUT_DIR, "metrics-base.csv"), index=False
)
metrics_per_grain_day_base.groupby(GRAIN_COL)["metric"].describe()

### 7.4  Visualize metrics

In this section we plot metric evolution over time for the best AutoML and the baseline models.

In [ ]:
metrics_df = metrics_per_grain_day.drop("time_series_id", axis=1).merge(
    metrics_per_grain_day_base.drop("time_series_id", axis=1),
    on=["metric_name", GRAIN_COL, PERIOD_COLUMN],
    how="inner",
    suffixes=["", "_base"],
)
metrics_df

In [ ]:
grain = [GRAIN_COL]
plot_filename = "metrics_plot.pdf"

pdf = PdfPages(os.path.join(os.getcwd(), OUTPUT_DIR, plot_filename))
for _, one_forecast in metrics_df.groupby(grain):
    one_forecast[PERIOD_COLUMN] = pd.to_datetime(one_forecast[PERIOD_COLUMN])
    one_forecast.sort_values(PERIOD_COLUMN, inplace=True)
    _draw_one_plot(
        one_forecast,
        PERIOD_COLUMN,
        target_column_name,
        grain,
        ["metric", "metric_base"],
        pdf,
        plot_predictions=True,
    )
pdf.close()

In [ ]:
from IPython.display import IFrame

IFrame(os.path.join("./forecast_output/metrics_plot.pdf"), width=800, height=300)

# 8. Inference

In this step, we generate an actual forecast by providing an inference set that does not contain actual values. This illustrates how to generate production forecasts in real life. The code in this section is pretty much identical to the one in section 6.1 with one exception, we set the `run_rolling_evaluation` argument to `False`.

### 8.1 Set up output dataset for inference data

Output of inference step can be represented as [OutputFileDatasetConfig](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.output_dataset_config.outputdatasetconfig?view=azure-ml-py) object which, in turn, will be registered as a dataset.

In [ ]:
from azureml.data import OutputFileDatasetConfig

output_inference_data_ds = OutputFileDatasetConfig(
    name="many_models_inference_output",
    destination=(datastore, "uci_electro_small/inference_output/"),
).register_on_complete(name="uci_electro_small_inference_data_ds")

In [ ]:
inference_dataset.take(5).to_pandas_dataframe()

For many models we need to provide the ManyModelsInferenceParameters object.

In [ ]:
from azureml.contrib.automl.pipeline.steps import AutoMLPipelineBuilder
from azureml.train.automl.runtime._many_models.many_models_parameters import (
    ManyModelsInferenceParameters,
)

output_file_name = "parallel_run_step.csv"
inference_ds_small = inference_dataset

mm_parameters = ManyModelsInferenceParameters(
    partition_column_names=time_series_id_column_names,
    time_column_name=time_column_name,
)

inference_steps = AutoMLPipelineBuilder.get_many_models_batch_inference_steps(
    experiment=experiment,
    inference_data=inference_ds_small,
    node_count=2,
    process_count_per_node=8,
    compute_target=compute_target,
    run_invocation_timeout=300,
    output_datastore=output_inference_data_ds,
    train_run_id=training_run.id,
    train_experiment_name=training_run.experiment.name,
    inference_pipeline_parameters=mm_parameters,
    append_row_file_name=output_file_name,
)

In [ ]:
from azureml.pipeline.core import Pipeline

inference_pipeline = Pipeline(ws, steps=inference_steps)
inference_run = experiment.submit(inference_pipeline)
inference_run.wait_for_completion(show_output=False)

In [ ]:
# from azureml.pipeline.core import Pipeline
# inference_pipeline = Pipeline(ws, [inference_step])
# inference_run = experiment.submit(inference_pipeline)
# inference_run.wait_for_completion(show_output=True)

### 8.2 Get the predicted data

In [ ]:
from azureml.contrib.automl.pipeline.steps.utilities import get_output_from_mm_pipeline

forecasting_results_name = "forecasting_results"
forecasting_output_name = "many_models_inference_output"
forecast_file = get_output_from_mm_pipeline(
    inference_run, forecasting_results_name, forecasting_output_name, output_file_name
)
inference_df = pd.read_csv(forecast_file)
print(
    "Prediction has ",
    inference_df.shape[0],
    " rows. Here the first 5 rows are being displayed.",
)
inference_df.head(5)

In [ ]:
inference_df.rename(columns={"Predictions": "predicted"}, inplace=True)
inference_df.to_csv(
    os.path.join(OUTPUT_DIR, "inference-set-predictions-many-models.csv"), index=False
)
inference_df.head(5)

## 9. Schedule Pipeline

This section is about how to schedule a pipeline for periodically predictions. For more info about pipeline schedule and pipeline endpoint, please follow this [notebook](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-setup-schedule-for-a-published-pipeline.ipynb).

In [ ]:
inference_published_pipeline = inference_pipeline.publish(
    name="automl_forecast_many_models",
    description="forecast many models",
    version="1",
    continue_on_step_failure=False,
)
print("Newly published pipeline id: {}".format(inference_published_pipeline.id))

If `inference_dataset` is going to refresh every 24 hours and we want to predict every 24 hours (forecast_horizon), we can schedule our pipeline to run every day at 11 pm to get daily inference results. You can refresh your test dataset (a newer version will be created) periodically when new data is available (i.e. target column in test dataset would have values in the beginning as context data, and followed by NaNs to be predicted). The inference pipeline will pick up context to further improve the forecast accuracy. See the <u><i>Forecasting away from training data</i></u> in this [notebook](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/automated-machine-learning/forecasting-forecast-function/auto-ml-forecasting-function.ipynb).

If `test_dataset` is going to refresh every 4 weeks before Friday 16:00 and we want to predict every 4 weeks (forecast_horizon), we can schedule our pipeline to run every 4 weeks at 16:00 to get daily inference results. You can refresh your test dataset (a newer version will be created) periodically when new data is available (i.e. target column in test dataset would have values in the beginning as context data, and followed by NaNs to be predicted). The inference pipeline will pick up context to further improve the forecast accuracy.

In [ ]:
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule

recurrence = ScheduleRecurrence(
    frequency="Day", interval=1, hours=[23], minutes=[00]  # Runs every day at 11:00 pm
)

schedule = Schedule.create(
    workspace=ws,
    name="many_models_inference_schedule",
    pipeline_id=inference_published_pipeline.id,
    experiment_name="schedule-run-mm-uci-electro",
    recurrence=recurrence,
    wait_for_provisioning=True,
    description="Schedule Run",
)

# You may want to make sure that the schedule is provisioned properly
# before making any further changes to the schedule

print("Created schedule with id: {}\n---".format(schedule.id))

### 9.1 [Optional] Disable schedule

In [ ]:
schedule.disable()